# Point Transformer
Cuda version


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


## Load Dataset

In [ ]:
!pip install open3d 1>/dev/null
%cd /content/gdrive/MyDrive/Developer/coarse-to-fine-shapes
# !git pull
from datasets.shapenet_data_pc import ShapeNet15kPointClouds
shapenet_location = "/content/gdrive/MyDrive/Developer/ShapeNetCore_pointclouds/ShapeNetCore.v2.PC15k"
dataset = ShapeNet15kPointClouds(root_dir=shapenet_location, tr_sample_size=1024, te_sample_size=1024)
from metrics.chamfer_distance import np_chamfer_distance

## Load Model

In [ ]:
%cd /content/gdrive/MyDrive/Developer
# !git clone https://github.com/POSTECH-CVLab/point-transformer.git
%cd point-transformer

/content/gdrive/MyDrive/Developer
/content/gdrive/MyDrive/Developer/point-transformer


In [ ]:
!conda install pytorch==1.9.0 torchvision -c pytorch

\ | / - \ | / - \ | / done
Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.
WARNING conda.models.version:get_matcher(546): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.9.0.*, but conda is ignoring the .* and treating it as 1.9.0
WARNING conda.models.version:get_matcher(546): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.8.0.*, but conda is ignoring the .* and treating it as 1.8.0
\ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [ ]:
!bash env_setup.sh pt

In [ ]:
import torch
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

In [ ]:
!conda env list

# conda environments:
#
base                     /usr/local
pt                       /usr/local/envs/pt



In [ ]:
%cd /content/gdrive/MyDrive/Developer
!git clone https://github.com/hszhao/PointWeb.git
%cd PointWeb
%cd lib/pointops
!python setup.py install

In [ ]:
%cd /content/gdrive/MyDrive/Developer/point-transformer
from model.pointtransformer.pointtransformer_seg import PointTransformerSeg, pointtransformer_seg_repro

model = pointtransformer_seg_repro()
train_losses = []
val_losses = []

## Overfit the model on a single pointcloud

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch

loss_fn = np_chamfer_distance

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
model.to(device)

for epoch in range(10):
  model.train(True)

  input = torch.unsqueeze(dataset[0]['train_points'], dim=0).to(device)
  output = torch.unsqueeze(dataset[0]['train_points'], dim=0).to(device)
  optimizer.zero_grad()
